In [50]:
from hydra import initialize, compose, utils
import polars as pl

In [51]:
%load_ext autoreload
%autoreload 2

from recs_utils.base_model import BaseRecommender
from recs_utils.load_data import MTSDataset
from recs_utils.split import TimeRangeSplit
from recs_utils.metrics import USER_ID_COL, ITEM_ID_COL, intra_list_diversity_hamming_per_user

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
with initialize("configs", version_base="1.3"):
    bm_25_config = compose("cross_val", ["model=bm25"])

In [53]:
with initialize("configs", version_base="1.3"):
    most_pop_config = compose("cross_val", ["model=most_pop"])

In [54]:
interactions = pl.read_parquet(bm_25_config.data.dump_files.interactions_path)
items = pl.read_parquet(bm_25_config.data.dump_files.items_path)
users = pl.read_parquet(bm_25_config.data.dump_files.users_path)

In [55]:
items = items.with_columns(pl.col("genres").cast(str).str.strip().str.to_lowercase().str.split(","))

In [56]:
bm25_rec: BaseRecommender = utils.instantiate(most_pop_config.model)

In [57]:
most_pop: BaseRecommender = utils.instantiate(bm_25_config.model)

In [58]:
dt_col = bm_25_config.cv.dt_column
last_date = interactions.get_column(dt_col).max()
folds = bm_25_config.cv.num_periods
interval = utils.instantiate(bm_25_config.cv.period)
start_date = last_date - interval * (folds + 1)
cv = TimeRangeSplit(start_date=start_date, interval=interval, folds=folds)

folds_with_stats = list(cv.split(
    interactions,
    user_column=USER_ID_COL,
    item_column=ITEM_ID_COL,
    datetime_column=dt_col,
    fold_stats=True
)
)

In [59]:
train_index, test_index, _ = folds_with_stats[0]

In [60]:
train_interactions = interactions.join(train_index, on=[USER_ID_COL, ITEM_ID_COL], how="inner")
test_interactions = interactions.join(test_index, on=[USER_ID_COL, ITEM_ID_COL], how="inner")

In [61]:
bm25_rec.fit(train_interactions)

In [62]:
most_pop.fit(train_interactions)

/mnt/2TBHDD/users/kernela/conda-envs/education-recsys/lib/python3.10/site-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.015803098678588867 seconds
  warnings.warn(


In [63]:
item_matrix = items.select(pl.col(ITEM_ID_COL, "genres")).explode("genres").with_columns(pl.lit(1).alias("val")).pivot(
    index=ITEM_ID_COL, columns="genres", values="val", aggregate_function="max").fill_null(0)

In [64]:
top_n = 10

In [65]:
item_matrix.head()

item_id,зарубежные детские книги,сказки,зарубежная классика,литература 19 века,русская классика,классическая проза,пьесы и драматургия,стихи и поэзия,зарубежная старинная литература,зарубежная драматургия,античная литература,литература 20 века,детские стихи,список школьной литературы 5-6 класс,политология,государственное и муниципальное управление,критика,мистика,детская проза,книги по экономике,список школьной литературы 10-11 класс,список школьной литературы 7-8 класс,книги про вампиров,общая история,литература 18 века,мифы / легенды / эпос,древнерусская литература,публицистика,исторические приключения,зарубежные приключения,зарубежный юмор,юмористическая проза,исторические любовные романы,классические любовные романы,зарубежная образовательная литература,биографии и мемуары,…,дистрибуция,прически / уход за волосами,математика 2 класс,математика 1 класс,русский язык 1 класс,бизнес-журналы,юридические журналы,событийный маркетинг,русский язык 2 класс,русский язык 3 класс,русский язык 4 класс,география 7 класс,обществознание 10 класс,обществознание 11 класс,обществознание 9 класс,орфографические словари,биографические справочники,чтение 4 класс,математика 3 класс,рабочие тетради / прописи,урология и нефрология,общественное питание,водный транспорт,школьные учебники по французскому языку,социокультурная деятельность,научная литература по психологии,мерчендайзинг,бродильные производства,гиа по географии (огэ,география 9 класс,гиа по иностранному языку (огэ,иностранный язык 9 класс,школьные учебники по английскому языку,биология 9 класс,гиа по химии (огэ,химия 9 класс,школьные учебники по музыке
u32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,…,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
128115,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
210979,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
95632,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
247906,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
294280,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Intra-List Diversity

In [66]:
def get_mean_intra_list_div(model: BaseRecommender, num_recs: int, item_matrix: pl.DataFrame):
    recs = model.recommend(test_interactions, num_recs_per_user=num_recs)
    return intra_list_diversity_hamming_per_user(recs, item_matrix).get_column("intra_list_div").mean()

In [67]:
get_mean_intra_list_div(most_pop, top_n, item_matrix)

0.004579829096088386

In [68]:
get_mean_intra_list_div(bm25_rec, top_n, item_matrix)

0.006760270334780216